## About

to create a notebook to make tf record for the bird , no bird model 

In [1]:
import os, json, random
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf, re, math
from tqdm import tqdm

## ff1010bird

In [8]:
DATATYPE = 'ff1010bird'
main_df = pd.read_csv('../data/ff1010bird/ff1010bird_metadata_2018.csv')
main_df.head()

,itemid,datasetid,hasbird,fold
0,64486,ff1010bird,0,5
1,2525,ff1010bird,0,1
2,44981,ff1010bird,0,4
3,101323,ff1010bird,0,4
4,165746,ff1010bird,0,5


In [7]:
from sklearn.model_selection import StratifiedKFold

SEED = 1
FOLDS = 8

skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=SEED)

main_df['fold'] = -1
for fold,(idxT,idxV) in enumerate( skf.split(main_df,main_df['hasbird'].values)):
    main_df['fold'][idxV] = fold

/tmp/ipykernel_462/3838190839.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  main_df['fold'][idxV] = fold
/tmp/ipykernel_462/3838190839.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  main_df['fold'][idxV] = fold
/tmp/ipykernel_462/3838190839.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  main_df['fold'][idxV] = fold
/tmp/ipykernel_462/3838190839.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of 

In [4]:
main_df.head()

,itemid,datasetid,hasbird,fold
0,64486,ff1010bird,0,5
1,2525,ff1010bird,0,1
2,44981,ff1010bird,0,4
3,101323,ff1010bird,0,4
4,165746,ff1010bird,0,5


## looks cool

In [4]:
print(main_df[main_df['fold']== 0]['hasbird'].value_counts())
print(main_df[main_df['fold']== 1]['hasbird'].value_counts())
print(main_df[main_df['fold']== 2]['hasbird'].value_counts())
print(main_df[main_df['fold']== 3]['hasbird'].value_counts())

0    720
1    242
Name: hasbird, dtype: int64
0    720
1    242
Name: hasbird, dtype: int64
0    720
1    241
Name: hasbird, dtype: int64
0    719
1    242
Name: hasbird, dtype: int64


In [8]:
main_df.to_csv('../data/ff1010bird/ff1010bird_metadata_2018.csv', index = False)

In [9]:
def get_fold(fold):
    val_df = main_df[main_df.fold==fold].reset_index(drop=True)
    return val_df

In [10]:
def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def serialize_example(audio,image_name,target):
    feature = {
        'image': _bytes_feature(audio),
        'image_name': _bytes_feature(image_name),
        'target': _int64_feature(target),
      }
    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()

In [11]:
def create_tf_records(fold = 0):
    df = get_fold(fold)
    tfr_filename = f'../data/tfrec/v1/happywhale-ff-2022-train-{fold}-{df.shape[0]}.tfrec'
    with tf.io.TFRecordWriter(tfr_filename) as writer:
        for i,row in tqdm(df.iterrows()):
            itemid = row.itemid
            target = row.hasbird
            image_path = f"../data/{DATATYPE}/audio/wav/{itemid}.wav"
            audio = tf.io.read_file(image_path)
            #audio, sample_rate = tf.audio.decode_wav(audio,
            #                                 desired_channels=-1,
            #                                 desired_samples=-1)
            image_name = str.encode(str(itemid))
            example = serialize_example(audio,image_name,target)
            writer.write(example)

In [12]:
for i in range(8):
    create_tf_records(fold = i)

962it [00:04, 202.23it/s]
962it [00:04, 208.04it/s]
961it [00:04, 203.47it/s]
961it [00:07, 130.98it/s]
961it [00:12, 73.96it/s] 
961it [00:04, 203.57it/s]
961it [00:04, 202.79it/s]
961it [00:06, 157.98it/s]


## warblrb10k_public

In [2]:
DATATYPE = 'warblrb10k_public'
main_df = pd.read_csv('../data/warblrb10k_public/warblrb10k_public_metadata_2018.csv')
main_df.head()

,itemid,datasetid,hasbird,fold
0,759808e5-f824-401e-9058,warblrb10k,1,1
1,1d94fc4a-1c63-4da0-9cac,warblrb10k,1,0
2,bb0099ce-3073-4613-8557,warblrb10k,1,7
3,c4c67e81-9aa8-4af4-8eb7,warblrb10k,1,4
4,ab322d4b-da69-4b06-a065,warblrb10k,0,4


In [3]:
from sklearn.model_selection import StratifiedKFold

SEED = 1
FOLDS = 8

skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=SEED)

main_df['fold'] = -1
for fold,(idxT,idxV) in enumerate( skf.split(main_df,main_df['hasbird'].values)):
    main_df['fold'][idxV] = fold

/tmp/ipykernel_700/3838190839.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  main_df['fold'][idxV] = fold
/tmp/ipykernel_700/3838190839.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  main_df['fold'][idxV] = fold
/tmp/ipykernel_700/3838190839.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  main_df['fold'][idxV] = fold
/tmp/ipykernel_700/3838190839.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of 

In [4]:
main_df.to_csv('../data/ff1010bird/warblrb10k_public_metadata_2018.csv', index = False)

In [3]:
def get_fold(fold):
    val_df = main_df[main_df.fold==fold].reset_index(drop=True)
    return val_df

def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def serialize_example(audio,image_name,target):
    feature = {
        'image': _bytes_feature(audio),
        'image_name': _bytes_feature(image_name),
        'target': _int64_feature(target),
      }
    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()

def create_tf_records(fold = 0):
    df = get_fold(fold)
    tfr_filename = f'../data/tfrec/v1/happywhale-war-2022-train-{fold}-{df.shape[0]}.tfrec'
    with tf.io.TFRecordWriter(tfr_filename) as writer:
        for i,row in tqdm(df.iterrows()):
            itemid = row.itemid
            target = row.hasbird
            image_path = f"../data/{DATATYPE}/audio/wav/{itemid}.wav"
            audio = tf.io.read_file(image_path)
            #audio, sample_rate = tf.audio.decode_wav(audio,
            #                                 desired_channels=-1,
            #                                 desired_samples=-1)
            image_name = str.encode(str(itemid))
            example = serialize_example(audio,image_name,target)
            writer.write(example)

In [ ]:
create_tf_records(fold = 0)

In [5]:
for i in range(8):
    create_tf_records(fold = i)

1000it [00:00, 1294.05it/s]
1000it [00:00, 1295.66it/s]
1000it [00:00, 1301.06it/s]
1000it [00:00, 1306.58it/s]
1000it [00:00, 1315.94it/s]
1000it [00:01, 967.06it/s]
1000it [00:02, 419.93it/s]
1000it [00:02, 429.19it/s]


## verify TF

In [20]:
def decode_audio(audio):
    audio = tf.audio.decode_wav(audio,)
    return audio.audio

def read_labeled_tfrecord(example):
    LABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
        "image_name": tf.io.FixedLenFeature([], tf.string),  # shape [] means single element
        'target': tf.io.FixedLenFeature([], tf.int64),
    }
    example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT)
    image = decode_audio(example['image'])
    label = example['target']
    return image, label # returns a dataset of (image, label) pairs

def load_dataset(filenames, labeled=True, ordered=False):
    # Read from TFRecords. For optimal performance, reading from multiple files at once and
    # disregarding data order. Order does not matter since we will be shuffling the data anyway.

    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False # disable order, increase speed

    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTO) # automatically interleaves reads from multiple files
    dataset = dataset.with_options(ignore_order) # uses data as soon as it streams in, rather than in its original order
    dataset = dataset.map(read_labeled_tfrecord)
    # returns a dataset of (image, label) pairs if labeled=True or (image, id) pairs if labeled=False
    return dataset

def get_training_dataset():
    dataset = load_dataset(TRAINING_FILENAMES, labeled=True)
    dataset = dataset.repeat() # the training dataset must repeat for several epochs
    dataset = dataset.shuffle(2048)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
    return dataset

def count_data_items(filenames):
    # the number of data items is written in the name of the .tfrec files, i.e. flowers00-230.tfrec = 230 data items
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in filenames]
    return np.sum(n)

In [21]:

AUTO = tf.data.experimental.AUTOTUNE
TRAINING_FILENAMES = tf.io.gfile.glob(f'../data/tfrec/v1/happywhale-2022-train*.tfrec')
print(len(TRAINING_FILENAMES))
dataset = load_dataset(TRAINING_FILENAMES, labeled=True)
dataset = dataset.repeat()
dataset = dataset.shuffle(2048)
dataset = dataset.batch(2)
dataset = dataset.prefetch(AUTO) #This dataset can directly be passed to keras.fit method
print(count_data_items(TRAINING_FILENAMES))

# Displaying single batch of TFRecord
train_batch = iter(dataset)
#display_batch_of_images(next(train_batch))

1
962


In [22]:
v = next(train_batch)

In [23]:
v[0]

<tf.Tensor: shape=(2, 441000, 1), dtype=float32, numpy=
array([[[-0.03469849],
        [-0.03753662],
        [-0.05050659],
        ...,
        [-0.03512573],
        [-0.04098511],
        [-0.04629517]],

       [[ 0.01980591],
        [ 0.02163696],
        [ 0.01907349],
        ...,
        [ 0.03814697],
        [ 0.03686523],
        [ 0.03713989]]], dtype=float32)>

In [24]:
v

(<tf.Tensor: shape=(2, 441000, 1), dtype=float32, numpy=
 array([[[-0.03469849],
         [-0.03753662],
         [-0.05050659],
         ...,
         [-0.03512573],
         [-0.04098511],
         [-0.04629517]],
 
        [[ 0.01980591],
         [ 0.02163696],
         [ 0.01907349],
         ...,
         [ 0.03814697],
         [ 0.03686523],
         [ 0.03713989]]], dtype=float32)>,
 <tf.Tensor: shape=(2,), dtype=int64, numpy=array([0, 0])>)